In [1]:
import os
from os.path import join as opj
import numpy as np
import random

import open3d as o3d
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt
import trimesh

from utils import *
from dataset import *

import pickle as pkl
data_root = 'data_root/3DGraspAff'
new_aff_path = opj(data_root, 'anntd_remapped_full_shape_train_data.pkl')
with open(new_aff_path, 'rb') as f:
    new_aff_dataset = pkl.load(f)
print("Loaded dataset with {} objects".format(new_aff_dataset['num_objects']))
print("Num of semantic classes: ", len(new_aff_dataset["semantic_classes"]))
print("Semantic classes: ", new_aff_dataset["semantic_classes"])
print("Num of affordances: ", len(new_aff_dataset["affordance_classes"]))
print("All affordances: ", new_aff_dataset["affordance_classes"])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.
Loaded dataset with 1096 objects
Num of semantic classes:  7
Semantic classes:  ['Knife', 'Bag', 'Bottle', 'Scissors', 'Mug', 'Earphone', 'Hat']
Num of affordances:  7
All affordances:  ['handover', 'cut', 'stab', 'lift', 'wrap', 'pour', 'wear']


In [2]:
# Save the meshes needed for the remapped dataset
from utils.partnet_utils import class_to_shapenet_dir, shapeid_to_partnet_dir
SHAPENET_PATH = '/home/sjauhri/IAS_WS/ShapeNetCore.v2'
PARTNET_PATH  = '/home/sjauhri/IAS_WS/PartNet/data_v0'

SHAPENET_SAVE_PATH = data_root + '/obj_meshes/train/ShapeNet'
# PARTNET_SAVE_PATH  = data_root + '/obj_meshes/train/PartNet_full'
PARTNET_SAVE_PATH  = data_root + '/obj_meshes/train/PartNet'

shapeid_to_partnet_dir_dict = dict()
# loop through the dataset
for sem_classs in new_aff_dataset['semantic_classes']:
    for shape_id in new_aff_dataset['data'][sem_classs].keys():
        
        try:
            shapeid_path = opj(SHAPENET_PATH, class_to_shapenet_dir[sem_classs.lower()],
                            shape_id)
            obj_path = opj(shapeid_path, 'models', 'model_normalized.obj')
            # check if obj_path exists
            if os.path.exists(obj_path):
                # try load with open3d
                # obj_mesh = o3d.io.read_triangle_mesh(obj_path)
                # load the texture separately
                # texture_path = opj(SHAPENET_PATH, class_to_shapenet_dir[sem_classs.lower()],
                #                     shape_id, 'models', 'model_normalized.mtl')

                # if succesful, just copy everything to the save path
                # save
                if not os.path.exists(opj(SHAPENET_SAVE_PATH, shape_id)):
                    os.makedirs(opj(SHAPENET_SAVE_PATH, shape_id))
                save_path = opj(SHAPENET_SAVE_PATH, shape_id)
                # copy everything from obj_path to save_path
                os.system('cp -r {} {}'.format(shapeid_path, save_path))
            else:
                print("ShapeNet category not found, try PartNet?")
        except:
            print("ShapeNet category not found, try PartNet?")
            # partnet_anno_id = shapeid_to_partnet_dir[shape_id]
            # meshes_path = opj(PARTNET_PATH, partnet_anno_id, 'objs')
            # # loop through all meshes in the folder and add to open3d object
            # obj_mesh = o3d.geometry.TriangleMesh()
            # for mesh in os.listdir(meshes_path):
            #     # optional: only load files with 'original' in the name
            #     mesh_path = opj(meshes_path, mesh)
            #     mesh = o3d.io.read_triangle_mesh(mesh_path)
            #     obj_mesh += mesh
            
            # # save
            # if not os.path.exists(opj(PARTNET_SAVE_PATH, partnet_anno_id)):
            #     os.makedirs(opj(PARTNET_SAVE_PATH, partnet_anno_id))
            # save_path = opj(PARTNET_SAVE_PATH, partnet_anno_id, 'model_normalized.obj')
            # o3d.io.write_triangle_mesh(save_path, obj_mesh, write_triangle_uvs=True)
        
        # viz
        # draw([obj_mesh])
        # break

ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try PartNet?
ShapeNet category not found, try P